In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
base_dir = "/content/drive/MyDrive/Data 298B Project Data/Test Dataset - Workbook 2"

# **Combining all of the dataframes**

In [4]:
metadata_df = pd.read_csv(f"{base_dir}/image_metadata.csv")

weather_df = pd.read_csv(f"{base_dir}/weather_data.csv")

modis_df = pd.read_csv(f"{base_dir}/remote_sensing_data.csv")

In [5]:
import time
start_time = time.time()

# Creating rounded latitude and longitude in metadata_df and modis_df for matching to the weather data (since latitude and longitude are rounded to 2 decimal places in weather_df)
metadata_df['Rounded_Latitude'] = metadata_df['Latitude'].round(2)
metadata_df['Rounded_Longitude'] = metadata_df['Longitude'].round(2)
modis_df['Rounded_Latitude'] = modis_df['Latitude'].round(2)
modis_df['Rounded_Longitude'] = modis_df['Longitude'].round(2)

# Ensuring the 'Date' columns are in the same format (YYYY-MM-DD)
metadata_df['Date'] = pd.to_datetime(metadata_df['Date']).dt.strftime('%Y-%m-%d')
weather_df['Date'] = pd.to_datetime(weather_df['Date']).dt.strftime('%Y-%m-%d')
modis_df['Date'] = pd.to_datetime(modis_df['Date']).dt.strftime('%Y-%m-%d')

# Merging metadata_df with weather_df
combined_part1_df = pd.merge(metadata_df, weather_df, left_on=['Rounded_Latitude', 'Rounded_Longitude', 'Date'], right_on=['Latitude', 'Longitude', 'Date'], how='left', suffixes=('', '_weather'))

# Dropping the rounded and duplicate columns from the weather_df merge
combined_part1_df.drop(columns=['Rounded_Latitude', 'Rounded_Longitude', 'Latitude_weather', 'Longitude_weather'], inplace=True)

# Merging the combined_df with modis_df based on the exact Latitude, Longitude, and Date match
df = pd.merge(combined_part1_df, modis_df, on=['Latitude', 'Longitude', 'Date'], how='left', suffixes=('', '_modis'))

# Dropping the rounded and duplicate columns from the final merge
df.drop(columns=['Rounded_Latitude', 'Rounded_Longitude'], inplace=True)

elapsed_time = time.time() - start_time
print(f"It took {elapsed_time:.4f} seconds to combine the image metadata, remote sensing data, and weather data.")

It took 0.0816 seconds to combine the image metadata, remote sensing data, and weather data.


In [6]:
df.head()

,Id,Latitude,Longitude,Date and Time,Date,Avg Temp 14d,Avg Humidity 14d,Total Precipitation 14d,Avg Wind Speed 14d,NDVI MODIS,NDVI - 1 MODIS,NDVI - 2 MODIS,EVI MODIS,EVI - 1 MODIS,EVI - 2 MODIS
0,IMG_1240.JPG,22.503033,120.485581,2019:01:26 13:42:38,2019-01-26,20.721429,73.842857,15.164,16.907143,0.3070,0.3511,0.5434,0.1609,0.2278,0.3503
1,IMG_1392.JPG,22.515956,120.487739,2019:01:26 15:07:09,2019-01-26,20.757143,73.835714,14.033,16.892857,0.3693,0.3120,0.5920,0.1913,0.1897,0.3809
2,IMG_1326.JPG,22.501131,120.487236,2019:01:26 14:14:18,2019-01-26,20.721429,73.842857,15.164,16.907143,0.2981,0.3343,0.5825,0.1584,0.2141,0.3810
3,IMG_20190308_114945.jpg,24.098481,120.669239,2019:03:08 11:49:45,2019-03-08,18.457143,80.807143,75.800,26.428571,0.3517,0.2443,0.2619,0.2528,0.1630,0.1510
4,IMG_20190308_111908.jpg,24.099251,120.668691,2019:03:08 11:19:07,2019-03-08,18.457143,80.807143,75.800,26.428571,0.3667,0.2737,0.2752,0.2687,0.1906,0.1694


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 0 to 19
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Id                       20 non-null     object 
 1   Latitude                 20 non-null     float64
 2   Longitude                20 non-null     float64
 3   Date and Time            20 non-null     object 
 4   Date                     20 non-null     object 
 5   Avg Temp 14d             20 non-null     float64
 6   Avg Humidity 14d         20 non-null     float64
 7   Total Precipitation 14d  20 non-null     float64
 8   Avg Wind Speed 14d       20 non-null     float64
 9   NDVI MODIS               20 non-null     float64
 10  NDVI - 1 MODIS           20 non-null     float64
 11  NDVI - 2 MODIS           20 non-null     float64
 12  EVI MODIS                20 non-null     float64
 13  EVI - 1 MODIS            20 non-null     float64
 14  EVI - 2 MODIS            20 

# **Adding Indicator Variables**

In [8]:
# Indicators for remote sensing data

# Adding the "NDVI 1 Decrease" column based on comparing "NDVI MODIS" and "NDVI - 1 MODIS"
df['NDVI 1 Decrease'] = np.where(df['NDVI MODIS'] < df['NDVI - 1 MODIS'], 1, 0)

# Adding the "NDVI 2 Decrease" column based on comparing "NDVI MODIS" and "NDVI - 2 MODIS"
df['NDVI 2 Decrease'] = np.where(df['NDVI MODIS'] < df['NDVI - 2 MODIS'], 1, 0)

# Adding the "EVI 1 Decrease" column based on comparing "EVI MODIS" and "EVI - 1 MODIS"
df['EVI 1 Decrease'] = np.where(df['EVI MODIS'] < df['EVI - 1 MODIS'], 1, 0)

# Adding the "EVI 2 Decrease" column based on comparing "EVI MODIS" and "EVI - 2 MODIS"
df['EVI 2 Decrease'] = np.where(df['EVI MODIS'] < df['EVI - 2 MODIS'], 1, 0)

In [9]:
df.head()

,Id,Latitude,Longitude,Date and Time,Date,Avg Temp 14d,Avg Humidity 14d,Total Precipitation 14d,Avg Wind Speed 14d,NDVI MODIS,NDVI - 1 MODIS,NDVI - 2 MODIS,EVI MODIS,EVI - 1 MODIS,EVI - 2 MODIS,NDVI 1 Decrease,NDVI 2 Decrease,EVI 1 Decrease,EVI 2 Decrease
0,IMG_1240.JPG,22.503033,120.485581,2019:01:26 13:42:38,2019-01-26,20.721429,73.842857,15.164,16.907143,0.3070,0.3511,0.5434,0.1609,0.2278,0.3503,1,1,1,1
1,IMG_1392.JPG,22.515956,120.487739,2019:01:26 15:07:09,2019-01-26,20.757143,73.835714,14.033,16.892857,0.3693,0.3120,0.5920,0.1913,0.1897,0.3809,0,1,0,1
2,IMG_1326.JPG,22.501131,120.487236,2019:01:26 14:14:18,2019-01-26,20.721429,73.842857,15.164,16.907143,0.2981,0.3343,0.5825,0.1584,0.2141,0.3810,1,1,1,1
3,IMG_20190308_114945.jpg,24.098481,120.669239,2019:03:08 11:49:45,2019-03-08,18.457143,80.807143,75.800,26.428571,0.3517,0.2443,0.2619,0.2528,0.1630,0.1510,0,0,0,0
4,IMG_20190308_111908.jpg,24.099251,120.668691,2019:03:08 11:19:07,2019-03-08,18.457143,80.807143,75.800,26.428571,0.3667,0.2737,0.2752,0.2687,0.1906,0.1694,0,0,0,0


In [10]:
df.to_csv(f"{base_dir}/combined_data.csv", index=False)